In [0]:
spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

%fs mkdirs /FileStore/GFK_data

%fs ls /FileStore/GFK_data

Initialize Spark session

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MovieRatings").getOrCreate()

Load data and Assign columnn names to data

In [0]:
movie_df = spark.read.csv("dbfs:/FileStore/GFK_data/movies.dat",header = False, sep= '::')
movie_df = movie_df.toDF("MovieID","Title","Genres")

ratings_df = spark.read.csv("dbfs:/FileStore/GFK_data/ratings.dat",header = False, sep= '::')
ratings_df = ratings_df.toDF("UserID","MovieID","Rating","Timestamp")

users_df = spark.read.csv("dbfs:/FileStore/GFK_data/users.dat", header = False, sep = '::')
users_df = users_df.toDF("UserID","Gender","Age","Occupation","Zip-Code")

shows loaded data

In [0]:
movie_df.show(10,truncate = False)
ratings_df.show(10,truncate = False)
users_df.show(10,truncate = False)

+-------+----------------------------------+----------------------------+
|MovieID|Title                             |Genres                      |
+-------+----------------------------------+----------------------------+
|1      |Toy Story (1995)                  |Animation|Children's|Comedy |
|2      |Jumanji (1995)                    |Adventure|Children's|Fantasy|
|3      |Grumpier Old Men (1995)           |Comedy|Romance              |
|4      |Waiting to Exhale (1995)          |Comedy|Drama                |
|5      |Father of the Bride Part II (1995)|Comedy                      |
|6      |Heat (1995)                       |Action|Crime|Thriller       |
|7      |Sabrina (1995)                    |Comedy|Romance              |
|8      |Tom and Huck (1995)               |Adventure|Children's        |
|9      |Sudden Death (1995)               |Action                      |
|10     |GoldenEye (1995)                  |Action|Adventure|Thriller   |
+-------+-----------------------------

importing require functions

In [0]:
from pyspark.sql.functions import split,col,avg,explode

filtering users by age 

In [0]:
filtered_user_df = users_df.filter((col("Age") >= 18) & (col("Age") <= 49))
filtered_user_df.show(10)

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-Code|
+------+------+---+----------+--------+
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
|     7|     M| 35|         1|   06810|
|     8|     M| 25|        12|   11413|
|     9|     M| 25|        17|   61614|
|    10|     F| 35|         1|   95370|
|    11|     F| 25|         1|   04093|
|    12|     M| 25|        12|   32793|
|    13|     M| 45|         1|   93304|
+------+------+---+----------+--------+
only showing top 10 rows



filtering movie by year (> 1989)

In [0]:
movie_df1 = movie_df.withColumn("Year",split(col("Title"),"\(").getItem(1).substr(1,4).cast("int"))
filtered_movie_df = movie_df1.filter(col("Year") > 1989)
filtered_movie_df.show(10)

+-------+--------------------+--------------------+----+
|MovieID|               Title|              Genres|Year|
+-------+--------------------+--------------------+----+
|      1|    Toy Story (1995)|Animation|Childre...|1995|
|      2|      Jumanji (1995)|Adventure|Childre...|1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|1995|
|      4|Waiting to Exhale...|        Comedy|Drama|1995|
|      5|Father of the Bri...|              Comedy|1995|
|      6|         Heat (1995)|Action|Crime|Thri...|1995|
|      7|      Sabrina (1995)|      Comedy|Romance|1995|
|      8| Tom and Huck (1995)|Adventure|Children's|1995|
|      9| Sudden Death (1995)|              Action|1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|1995|
+-------+--------------------+--------------------+----+
only showing top 10 rows



Joining dataframes

In [0]:
joined_df = ratings_df.join(filtered_user_df,"UserID").join(filtered_movie_df,"MovieID")
joined_df.show(10,truncate = False)

+-------+------+------+---------+------+---+----------+--------+---------------------------+------------------------------+----+
|MovieID|UserID|Rating|Timestamp|Gender|Age|Occupation|Zip-Code|Title                      |Genres                        |Year|
+-------+------+------+---------+------+---+----------+--------+---------------------------+------------------------------+----+
|1641   |3     |2     |978298430|M     |25 |15        |55117   |Full Monty, The (1997)     |Comedy                        |1997|
|648    |3     |3     |978297867|M     |25 |15        |55117   |Mission: Impossible (1996) |Action|Adventure|Mystery      |1996|
|3534   |3     |3     |978297068|M     |25 |15        |55117   |28 Days (2000)             |Comedy                        |2000|
|104    |3     |4     |978298486|M     |25 |15        |55117   |Happy Gilmore (1996)       |Comedy                        |1996|
|1431   |3     |3     |978297095|M     |25 |15        |55117   |Beverly Hills Ninja (1997) |Actio

Handle genres

In [0]:
gen_df = joined_df.withColumn("Genre",explode(split(col("Genres"),"\\|")))
gen_df.show(10,truncate = False)

+-------+------+------+---------+------+---+----------+--------+---------------------------+------------------------+----+---------+
|MovieID|UserID|Rating|Timestamp|Gender|Age|Occupation|Zip-Code|Title                      |Genres                  |Year|Genre    |
+-------+------+------+---------+------+---+----------+--------+---------------------------+------------------------+----+---------+
|1641   |3     |2     |978298430|M     |25 |15        |55117   |Full Monty, The (1997)     |Comedy                  |1997|Comedy   |
|648    |3     |3     |978297867|M     |25 |15        |55117   |Mission: Impossible (1996) |Action|Adventure|Mystery|1996|Action   |
|648    |3     |3     |978297867|M     |25 |15        |55117   |Mission: Impossible (1996) |Action|Adventure|Mystery|1996|Adventure|
|648    |3     |3     |978297867|M     |25 |15        |55117   |Mission: Impossible (1996) |Action|Adventure|Mystery|1996|Mystery  |
|3534   |3     |3     |978297068|M     |25 |15        |55117   |28 Da

Calculating average rating per genre per year

In [0]:
avg_ratings_df = gen_df.groupBy("Genre","Year").agg(avg("Rating").alias("AvgRating"))
avg_ratings_df.show(10,truncate = False)

+-----------+----+------------------+
|Genre      |Year|AvgRating         |
+-----------+----+------------------+
|Horror     |1990|3.1979010494752624|
|Documentary|1997|3.8688212927756656|
|Romance    |1998|3.4995221712538225|
|Children's |1992|3.259950248756219 |
|Children's |1998|3.383314020857474 |
|Mystery    |1997|3.5251304586652017|
|Animation  |1995|3.9835735836406303|
|Adventure  |1996|3.2408491947291362|
|Documentary|1996|4.10814606741573  |
|Horror     |1994|3.0444659255679074|
+-----------+----+------------------+
only showing top 10 rows



%fs mkdirs /FileStore/GFK_data/results

saving results 

In [0]:
avg_ratings_df.write.csv("dbfs:/FileStore/GFK_data/results",header= True, mode = "Overwrite")

In [0]:
%fs ls /FileStore/GFK_data/results

path,name,size,modificationTime
dbfs:/FileStore/GFK_data/results/_SUCCESS,_SUCCESS,0,1724846293000
dbfs:/FileStore/GFK_data/results/_committed_7236810653229926080,_committed_7236810653229926080,112,1724846293000
dbfs:/FileStore/GFK_data/results/_started_7236810653229926080,_started_7236810653229926080,0,1724846292000
dbfs:/FileStore/GFK_data/results/part-00000-tid-7236810653229926080-b01d1c8c-151a-44e3-8303-9965cb5ac8c3-54-1-c000.csv,part-00000-tid-7236810653229926080-b01d1c8c-151a-44e3-8303-9965cb5ac8c3-54-1-c000.csv,5888,1724846293000
